In [3]:
import os
from pathlib import Path
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine

In [4]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [3]:
db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

with engine.connect() as conn, conn.begin():  
    
    ultimo_costo_sae = pd.read_sql_table('ultimo_costo_sae', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [21]:
pd.Timedelta(hours=7)

Timedelta('0 days 07:00:00')

In [6]:
def search_costo_ventas_func(mes: int) -> list[int, str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    

    param_dia_ini = pd.Timestamp(2024, mes, 1)
    param_dia_fin = datetime(2024, mes + 1, 1) - timedelta(days= 1)

    param_dia_hr_ini = param_dia_ini + timedelta(hours=7)
    param_dia_hr_fin = param_dia_fin + timedelta(hours=31) - timedelta(seconds= 1)
    
    search_costo_vasdfasdfsadentas = [
        "&",
            ("payment_date", ">=", param_dia_hr_ini.strftime('%Y-%m-%d %H:%M:%S')),
            ("payment_date", "<=", param_dia_hr_fin.strftime('%Y-%m-%d %H:%M:%S')),
        ]
        
    # De 01/06/2024 00:00:00 al 31/06/2024 23:59:59"
    search_costo_ventas = [
        "&", "&",
        ("state", "in", ["purchase", "done"]),
        ("date_approve", ">=", "2024-06-01 07:00:00"),
        ("date_approve", "<=", "2024-07-01 06:59:59")
    ]

    return mes, search_costo_ventas

# Pruebas

In [14]:
api_params = api_params_func()
search_costo_ventas = search_costo_ventas_func(1)